<a href="https://colab.research.google.com/github/StarBlossom99/Pytorch_Study/blob/main/2022_0118/pytorch_study_parameter_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


In [4]:
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad() # 이전 batch 에서의 gradient 값을 초기화
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): # Test를 돌리는 것이므로 gradient를 바꿀 필요가 없어서 no grad() 블록 안에서 실행시킨다.
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for 문에 있는 (X,y)는 dataloader 안에 들어있는 Data 와 Label의 쌍이고 X는 Data, y는 Label을 의미하게 된다.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    #각 epoch 마다 train 한번 test 한번씩 돌리면서 결과를 프린트
print("Done!")

Epoch 1
-------------------------------
loss: 2.312450  [    0/60000]
loss: 2.300748  [ 6400/60000]
loss: 2.282207  [12800/60000]
loss: 2.262926  [19200/60000]
loss: 2.251174  [25600/60000]
loss: 2.220835  [32000/60000]
loss: 2.224730  [38400/60000]
loss: 2.196404  [44800/60000]
loss: 2.188159  [51200/60000]
loss: 2.145602  [57600/60000]
Test Error: 
 Accuracy: 43.5%, Avg loss: 2.150801 

Epoch 2
-------------------------------
loss: 2.171952  [    0/60000]
loss: 2.159152  [ 6400/60000]
loss: 2.099385  [12800/60000]
loss: 2.101493  [19200/60000]
loss: 2.044114  [25600/60000]
loss: 1.992452  [32000/60000]
loss: 2.009165  [38400/60000]
loss: 1.936981  [44800/60000]
loss: 1.938894  [51200/60000]
loss: 1.847862  [57600/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 1.858555 

Epoch 3
-------------------------------
loss: 1.910127  [    0/60000]
loss: 1.872175  [ 6400/60000]
loss: 1.755075  [12800/60000]
loss: 1.779739  [19200/60000]
loss: 1.657442  [25600/60000]
loss: 1.632836  [32000/600